<a href="https://colab.research.google.com/github/samuel0711/Desafio/blob/main/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
import psycopg2
import datetime as dt

import matplotlib.pyplot as plt 

pd.set_option('display.max_columns', 50)

# Criando funções de auxílio

In [43]:
# Função para criar conexão no banco
def conecta_db():
  con = psycopg2.connect(host='ec2-3-222-74-92.compute-1.amazonaws.com', 
                         database='dfo2vqe9hs2knm',
                         user='zgbodaynfjorzt',
                         port='5432', 
                         password='d2b11c0c6039df67f39d22f680fb92a843a1b652e3d5b692f6670c0a2abc2804')
  return con

In [44]:
#Consulta em tabela específica
def pegar_tabela(tabela):
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  #construindo query
  query = 'SELECT * FROM '+tabela
  cur.execute(query)
  # pegar todos os dados da query e armazenar dentro de uma lista
  data = cur.fetchall()

  registros = []
  for rec in data:
    registros.append(rec)
  con.close()

  #print(registros[0])
  return registros

#Consulta sobre nomes das colunas de uma tabela
def pegar_colunas(tabela):
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  #construindo query
  query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = '"+tabela+"'"
  cur.execute(query)
  col_names = cur.fetchall()
  con.close()

  col_names = [col[0] for col in col_names]
  #print(col_names)
  return col_names

# Função para consulta de versão e tabelas do banco
def consultar_db():
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  # executa uma declaração
  print('Versão do banco de dados PostgreSQL:')
  cur.execute ('SELECT version()')
  # exibir a versão do servidor de banco de dados PostgreSQL
  db_version = cur.fetchone()
  print(db_version)

  #verificando tabelas disponíveis
  cur.execute ("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema'")
  tables = cur.fetchall()
  for rec in tables:
    print(rec)

#Retorna as consultas e monta um dataframe
def fazer_consulta(tabela):
  registros = pegar_tabela(tabela)
  colunas = pegar_colunas(tabela)
  df = pd.DataFrame(registros, columns=colunas)
  return df

In [45]:
consultar_db()

Versão do banco de dados PostgreSQL:
('PostgreSQL 14.4 (Ubuntu 14.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',)
('public', 'forms', 'zgbodaynfjorzt', None, True, False, True, False)
('public', 'order_items', 'zgbodaynfjorzt', None, True, False, True, False)
('public', 'orders', 'zgbodaynfjorzt', None, True, False, True, False)


In [46]:
forms = fazer_consulta('forms')
order_items = fazer_consulta('order_items')
orders = fazer_consulta('orders')

In [47]:
print(forms.shape, order_items.shape, orders.shape) #order items = orders, porém com granularidade de produtos por registro

(427950, 23) (211663, 6) (55078, 12)


# Exploração dos dados

In [48]:
orders.loc[(orders.id_cliente == 'b6c0b82cc271fd1203fc') & (orders.status_pagamento != 'Cancelado')].sort_values(by='data_pedido')

,id_pedido,id_cliente,data_pedido,status_pagamento,tipo_pagamento,valor_total,valor_desconto,frete_gratis,cidade,estado,locale,utm_source
35809,3de41ce9e954cf4,b6c0b82cc271fd1203fc,2022-05-16 09:01:58,Pagamento aprovado,CARTÃO,274.25,0.00,False,São Paulo,SP,BR,None


In [49]:
order_items[order_items.id_pedido == '3de41ce9e954cf4']

,id,id_pedido,id_form,id_produto,qtde,valor
191159,614962,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,d2e0,1,64.90
191160,614963,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,509d,1,74.95
191161,614964,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,e820,1,74.95
191162,614965,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,d346,1,59.90


In [50]:
forms[forms.id_form=='cca2dd772b2989216d22bf0d422c25']

,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale,cidade,estado,qtde_lavagem_semana,utm_source,tipo_cabelo,comprimento,tipo_fios,procedimentos,tempo_procedimento,dieta,atividade_fisica,frequencia_estresse,caracteristica,faixa_etaria,efeitos_desejados,fragancia
76881,cca2dd772b2989216d22bf0d422c25,b6c0b82cc271fd1203fc,2022-01-26 08:36:49,iOS,Safari,"Ribeirão Preto, Sao Paulo",BR,RIBEIRÃO PRETO,SAO PAULO,4,None,Tipo 3,Longo,Perfeitos,Exposição ao sol,Não houve procedimentos,Leve,Raramente,Nunca,Característica 6,25 a 34,Efeito 2,Rosas


# Criando label de formulários que se transformaram em venda

In [51]:
sucess_lead = ['Entregue', 'Pagamento aprovado', 'Em transporte']
df_orders = orders[orders.status_pagamento.isin(sucess_lead)]

sucess_orders = df_orders.id_pedido.to_list()
df_order_items = order_items[order_items.id_pedido.isin(sucess_orders)]
sucess_forms = df_order_items.id_form.to_list()

forms['lead_into_sale'] = np.where(forms.id_form.isin(sucess_forms), True, False)
forms.lead_into_sale.value_counts()

False    382189
True      45761
Name: lead_into_sale, dtype: int64

In [52]:
forms_y = forms['lead_into_sale']
forms.drop(columns='lead_into_sale', inplace=True)